## From kaggle
https://www.kaggle.com/fmarazzi/baseline-keras-cnn-roc-fast-10min-0-925-lb

In [1]:
'''For preprocessing images'''
import numpy as np
from PIL import Image
import scipy
import matplotlib.pyplot as plt
import csv
import glob
'''For CNN'''
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import MobileNet
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
from matplotlib.pyplot import imshow

Using TensorFlow backend.


In [12]:
image_size = 96
train_data_size = 4000
test_data_size = 1000

In [13]:
def standardize(img):
    # padding
    longer_side = max(img.size)
    horizontal_padding = (longer_side - img.size[0]) / 2
    vertical_padding = (longer_side - img.size[1]) / 2
    img = img.crop(
        (
            -horizontal_padding,
            -vertical_padding,
            img.size[0] + horizontal_padding,
            img.size[1] + vertical_padding
        )
    )
    # resizing to standardized size
    img = img.resize([image_size,image_size],Image.ANTIALIAS) \
    # plt.imshow(img) # To see the image being standardized.
    
    # converting image to numpy array
    img.load()
    img = np.asarray(img, dtype="int32")
    return img

In [14]:
def function():
    for filename in glob.glob('input/subset_data/train/*.tif'):
        img =Image.open(filename)
        img = standardize(img)
        print(img.shape)
        return

In [15]:
function()

(96, 96, 3)


In [16]:
'''Loading data'''
def get_id_from_filename(filename):
    id = filename.split("/")[-1]
    id = id.split(".")[0]
    return id

In [17]:
def load_train():
    names = []
    # Change first number base on number of training examples
    X_train = np.empty((train_data_size,image_size,image_size,3), dtype="int32")
    Y_train = np.empty(shape=(train_data_size,2),dtype="int32")

    i = 0
    for filename in glob.glob('input/subset_data/train/*.tif'):
        names.append(get_id_from_filename(filename))
        img =Image.open(filename)
        img = standardize(img)
        X_train[i-1] = img
        i += 1
        
    with open('input/subset_data/train_labels_full.csv') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        next(readCSV, None)
        for row in readCSV:
            name = row[0]
            if name in names:
                label = int(row[1])
                if label == 0:
                    Y_train[names.index(name)] = np.array([1,0]) # means 0
                elif label == 1:
                    Y_train[names.index(name)] = np.array([0,1]) # means 1
    return X_train,Y_train

In [18]:
def load_test():
    names = []
    # Change first number base on number of training examples
    X_test = np.empty((test_data_size,image_size,image_size,3), dtype="int32")
    Y_test = np.empty(shape=(test_data_size,2),dtype="int32")

    i = 0
    for filename in glob.glob('input/subset_data/test_with_outputs/*.tif'):
        names.append(get_id_from_filename(filename))
        img =Image.open(filename)
        img = standardize(img)
        X_test[i-1] = img
        i += 1
        
    with open('input/subset_data/train_labels_full.csv') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        next(readCSV, None)
        for row in readCSV:
            name = row[0]
            if name in names:
                label = int(row[1])
                if label == 0:
                    Y_test[names.index(name)] = np.array([1,0]) # means 0
                elif label == 1:
                    Y_test[names.index(name)] = np.array([0,1]) # means 1
    return X_test,Y_test

In [19]:
X_train_orig,Y_train_orig = load_train()
X_test_orig,Y_test_orig = load_test()

# Normalizing for faster convergence
X_train = X_train_orig/255.
X_test = X_test_orig/255.
Y_train = Y_train_orig
Y_test = Y_test_orig

print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)
print("X_test shape: ", X_test.shape)
print("Y_test shape: ", Y_test.shape)

# To check values inside.
# print(X_train)
# print(Y_train)
# print(X_test)
# print(Y_test)

X_train shape:  (4000, 96, 96, 3)
Y_train shape:  (4000, 2)
X_test shape:  (1000, 96, 96, 3)
Y_test shape:  (1000, 2)


## Define the model
### Model structure (optimizer: Adam):

- Input
- [Conv2D*3 -> MaxPool2D -> Dropout] x3 --> (filters = 16, 32, 64)
- Flatten
- Dense (256)
- Dropout
- Output

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam

kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

dropout_conv = 0.3
dropout_dense = 0.5

model = Sequential()
model.add(Conv2D(first_filters, kernel_size, activation = 'relu', input_shape = (image_size, image_size, 3)))
model.add(Conv2D(first_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

#model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(256, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(dropout_dense))
# model.add(Dense(1, activation = "sigmoid"))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(Adam(0.01), loss = "binary_crossentropy", metrics=["accuracy"])

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 94, 94, 32)        896       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 92, 92, 32)        9216      
_________________________________________________________________
batch_normalization_13 (Batc (None, 92, 92, 32)        128       
_________________________________________________________________
activation_13 (Activation)   (None, 92, 92, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 46, 46, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 46, 46, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 44, 44, 64)        18432     
__________

In [23]:
model.fit(X_train, Y_train, batch_size = 32, epochs = 13, verbose = 1, validation_data=(X_test,Y_test))

Instructions for updating:
Use tf.cast instead.
Train on 4000 samples, validate on 1000 samples
Epoch 1/13
4000/4000 [==============================] - 13s 3ms/step - loss: 0.8613 - acc: 0.5473 - val_loss: 1.0673 - val_acc: 0.5850
Epoch 2/13
4000/4000 [==============================] - 9s 2ms/step - loss: 0.7473 - acc: 0.5555 - val_loss: 0.7508 - val_acc: 0.5520
Epoch 3/13
4000/4000 [==============================] - 9s 2ms/step - loss: 0.7191 - acc: 0.5635 - val_loss: 0.9722 - val_acc: 0.5020
Epoch 4/13
4000/4000 [==============================] - 9s 2ms/step - loss: 0.7114 - acc: 0.5635 - val_loss: 0.6715 - val_acc: 0.6040
Epoch 5/13
4000/4000 [==============================] - 9s 2ms/step - loss: 0.6925 - acc: 0.5740 - val_loss: 0.6731 - val_acc: 0.6030
Epoch 6/13
4000/4000 [==============================] - 9s 2ms/step - loss: 0.6914 - acc: 0.5717 - val_loss: 0.6712 - val_acc: 0.6070
Epoch 7/13
4000/4000 [==============================] - 9s 2ms/step - loss: 0.6829 - acc: 0.5875 - 